In [1]:
# Количество и название подключенного GPU
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-125b3888-9730-78a5-c3ee-54cb2c71495f)


In [ ]:
#Установка Keras версии 2.3.1
!pip install Keras==2.3.1

In [ ]:
# Печать версии Keras. Должна быть именно 2.3.1
!python -c 'import keras; print(keras.__version__)'

Using TensorFlow backend.
2020-07-08 14:28:01.092068: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2.3.1


 ## Скачиваем и извлекаем набор данных

In [ ]:
import os

In [ ]:
# заменить на свое Kaggle-имя и ключ
os.environ['KAGGLE_USERNAME'] = "katerindanilina" 
os.environ['KAGGLE_KEY'] = "XXX" 

In [ ]:
# Скачиваем и разархивируем
!kaggle datasets download kbhartiya83/swimming-pool-and-car-detection -p /content/data --unzip 

 91% 101M/111M [00:01<00:00, 73.1MB/s] 
100% 111M/111M [00:01<00:00, 62.5MB/s]


## Клонируем хранилище Github c RetinatNet

In [ ]:
repo_url = 'https://github.com/fizyr/keras-retinanet'
repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url))) 

!git clone {repo_url} # клонируем хранилище в ОП ноутбука Colab

Cloning into 'keras-retinanet'...
remote: Enumerating objects: 5862, done.
remote: Total 5862 (delta 0), reused 0 (delta 0), pack-reused 5862
Receiving objects: 100% (5862/5862), 13.40 MiB | 8.28 MiB/s, done.
Resolving deltas: 100% (3948/3948), done.


In [ ]:
# Переходим в папку /content/keras-retinanet
%cd {repo_dir_path} 
!pip install .
!python setup.py build_ext --inplace

%cd ..

/content/keras-retinanet
Processing /content/keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-0.5.1-cp36-cp36m-linux_x86_64.whl size=170780 sha256=24f5c1f04fbf1236da44a4480f772c03d0f3420e44dfcab010c1324e6df759ea
  Stored in directory: /root/.cache/pip/wheels/b2/9f/57/cb0305f6f5a41fc3c11ad67b8cedfbe9127775b563337827ba
  Created wheel for keras-resnet: filename=keras_resnet-0.1.0-py2.py3-none-any.whl size=13346 sha256=2e7feb62ceccac1bd5d3a0e1f6f9252a62d6240374407722bd87179763d8ef64
  Stored in directory: /root/.cache/pip/wheels/80/dd/ac/842235b63dddac12faa4b48ebe58b8944e8c2e57c2e38dddb6
Successfully built keras-retinanet keras-resnet
running build_ext
cythoning keras_retinanet/utils/compute_overlap.pyx to keras_retinanet/utils/compute_overlap.c
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/keras-retinanet/k

## Клонируем репозиторий ije_retinanet

In [ ]:
ije_repo_url = 'https://github.com/ijeism/ije_retinanet'
ije_repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(ije_repo_url))) 

!git clone {ije_repo_url} # клонируем репозиторий
# %cd {ije_repo_dir_path} 
# !git pull # pull repo

Cloning into 'ije_retinanet'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 78 (delta 39), reused 41 (delta 15), pack-reused 0
Unpacking objects: 100% (78/78), done.


## Подготовка набора данных для обучения

In [ ]:
import os
base_dir = os.getcwd()
base_dir

'/content'

In [ ]:
from shutil import copyfile
import random

In [ ]:
# Выберем подвыборку размером N для первого обучения
# determine number for subsample
NUM = 800

# Создаем папку для подвыборки
sub_dir = base_dir + '/images_subset/'
!mkdir {sub_dir}

image_dir = base_dir + '/data/training_data/training_data/images/'
image_paths = os.listdir(image_dir)
random_NUM = random.sample(image_paths, NUM) # Рандомно выбираем 800 изображений

# Копируем подвыборку в папку
for i in random_NUM:
    copyfile(image_dir + i, sub_dir + i )

In [ ]:
# Проверим количество изображений в папке подвыборки
%cd /content/images_subset/
!ls -F |grep -v / | wc -l

/content/images_subset
800


In [ ]:
# Создаем необходимые файлы для обучения
!python /content/ije_retinanet/build_dataset.py \
-l /content/data/training_data/training_data/labels/ \
-i /content/images_subset/ \
-r /content/images_subset/train.csv \
-e /content/images_subset/test.csv \
-c /content/images_subset/classes.csv 

[INFO] creating 'train' set ...
[INFO] 640 total images in 'train'
[INFO] writing train annotations ...
[INFO] total 2509 annotations
[INFO] train.csv completed
[INFO] creating 'test' set ...
[INFO] 160 total images in 'test'
[INFO] writing test annotations ...
[INFO] total 593 annotations
[INFO] test.csv completed
[INFO] writing classes ...
[INFO] classes.csv completed
[FINAL] Task completed!


In [ ]:
# ПРоверяем созданные файлы
!ls -l

## Скачиваем предварительно обученную модель

In [ ]:
import urllib.request

In [ ]:
# путь, по которому должны быть сохранены предварительно обученные веса 
PRETRAINED_MODEL = '/content/keras-retinanet/snapshots/resnet50_coco_best_v2.1.0.h5'
# url модели
URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
# извлечь модель по URL модели и сохранить
urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)

('/content/keras-retinanet/snapshots/resnet50_coco_best_v2.1.0.h5',
 <http.client.HTTPMessage at 0x7fbef16eeef0>)

## Обучение CNN

In [ ]:
# Считаем количество аннотаций в файле train.csv
!wc -l /content/images_subset/train.csv

2509 /content/images_subset/train.csv


In [ ]:
# вычисляем количество шагов на одну эпоху из расчета: annotations/batch-size

import pandas as pd
import math

df = pd.read_csv('/content/images_subset/train.csv', header=None)
count = len(df)
no_steps = math.ceil(count/32)

print("Count of annotations: {}".format(count))
print("Number of steps per epoch: {}".format(no_steps))

Count of annotations: 2509
Number of steps per epoch: 79


In [ ]:
# Создаем отдельную папку для первого обучения и переходим в директорию
!mkdir /content/run_1/
%cd /content/run_1/

/content/run_1


In [ ]:
!python /content/keras-retinanet/keras_retinanet/bin/train.py \
--freeze-backbone \
--random-transform \
--weights {PRETRAINED_MODEL} \
--weighted-average \
--batch-size 32 \
--steps {no_steps} \
--epochs 30 \
csv '/content/images_subset/train.csv' '/content/images_subset/classes.csv'

Using TensorFlow backend.
2020-07-08 15:13:28.663031: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Creating model, this may take a second...
2020-07-08 15:13:31.309882: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-07-08 15:13:31.367223: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-08 15:13:31.368091: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-07-08 15:13:31.368155: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
20

/content/run_1/snapshots/resnet50_csv_30.h5


## Оценка модели

In [ ]:
# Путь к папке весов переобученной модели
model_1_path = os.path.join('/content/run_1/snapshots/', sorted(os.listdir('/content/run_1/snapshots/'), reverse=True)[0])
print(model_1_path)

In [ ]:
# Оценка модели на тестовом наборе данных test.csv
!python /content/keras-retinanet/keras_retinanet/bin/evaluate.py \
csv '/content/images_subset/test.csv' \
'/content/images_subset/classes.csv' \
{model_1_path} --convert-model

%cd ..

/content/run_1/snapshots/resnet50_csv_30.h5
Using TensorFlow backend.
2020-07-08 19:52:24.343533: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Loading model, this may take a second...
2020-07-08 19:52:31.027411: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-07-08 19:52:31.052427: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-08 19:52:31.053156: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-07-08 19:52:31.053201: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully 

## Генерируем боксы обнаружения на тестовых изображениях -  файлы аннотации

In [ ]:
# Создаем директорию с выбранными естовыми изображениями 

!mkdir /content/data/test_data_images/test_data_images/test
!mv -t /content/data/test_data_images/test_data_images/test \
/content/data/test_data_images/test_data_images/images/000000091.jpg \
/content/data/test_data_images/test_data_images/images/000000097.jpg \
/content/data/test_data_images/test_data_images/images/000002576.jpg \
/content/data/test_data_images/test_data_images/images/000002412.jpg \
/content/data/test_data_images/test_data_images/images/000000076.jpg \
/content/data/test_data_images/test_data_images/images/000000441.jpg

## Конвертируем переобученную модель в файл весов

In [ ]:
# Напечатаем путь до переобученной модели
model_1_path

'/content/run_1/snapshots/resnet50_csv_30.h5'

In [ ]:
os.getcwd()

'/content'

In [ ]:
# преобразуем переобученную модель в файл весов, который будет использоваться для генерации прогнозов
!python /content/keras-retinanet/keras_retinanet/bin/convert_model.py '/content/run_1/snapshots/resnet50_csv_30.h5' '/content/run_1/snapshots/resnet50_csv_30_inference.h5'
inference_model = '/content/run_1/snapshots/resnet50_csv_30_inference.h5'

Using TensorFlow backend.
2020-07-08 19:53:38.379117: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-07-08 19:53:40.197484: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-07-08 19:53:40.217810: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-08 19:53:40.218649: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-07-08 19:53:40.218701: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-07-08 19:53:40.220929: I tensorflow/str

In [ ]:
# Генерируем файл с прогнозами

!python /content/ije_retinanet/image_inference_write.py \
-i '/content/data/test_data_images/test_data_images/test/' \
-t 0.6 \
-m {inference_model} \
-o  /content/data/ \
-l /content/images_subset/classes.csv

Using TensorFlow backend.
2020-07-08 19:54:17.071687: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-07-08 19:54:19.614282: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-07-08 19:54:19.633007: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-08 19:54:19.634015: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-07-08 19:54:19.634067: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-07-08 19:54:19.636377: I tensorflow/str

In [ ]:
import glob

In [ ]:
# Соединяем все файлы предсказаний в один

fout=open("/content/data/output/out.csv","a")

fout.write("path,confidence,ymin,xmin,ymax,xmax,class\n") # Заголовок файла
# добавление файлов 
for fi in glob.glob("/content/data/*.csv"):
  f = open(fi)
  for line in f:
    fout.write(line)
fout.close()

## Генерируем боксы обнаружения на тестовых изображениях -  изображения

In [ ]:
os.getcwd()

'/content'

In [ ]:
# создайте выходной каталог, в котором вы хотите сохранить изображения с помощью ограничительных рамок 
!mkdir /content/data/output

In [ ]:
# Генерируем ограничительные рамки на изображениях
!python /content/ije_retinanet/image_inference_print.py \
-i /content/data/test_data_images/test_data_images/test/ \
-t 0.6 \
-m  {inference_model} \
-o /content/data/output

mkdir: cannot create directory ‘/content/data/output’: File exists
Using TensorFlow backend.
2020-07-08 19:56:42.193648: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-07-08 19:56:44.448735: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-07-08 19:56:44.468820: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-08 19:56:44.469532: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-07-08 19:56:44.469611: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic lib

In [ ]:
from google.colab import files

# Скачиваем изображения - результат детекции
for p in glob.glob('/content/data/output/*.jpg'):
  files.download(p)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
  # Скачиваем исходные изображения, выбранные для теста
for p in glob.glob('/content/data/test_data_images/test_data_images/test/*.jpg'):
  files.download(p)

In [ ]:
# download файл весов переобученной модели
for p in glob.glob('/content/run_1/snapshots/resnet50_csv_30_inference.h5'):
  files.download(p)

In [ ]:
# создать dataframe для визуализации результатов детекции - аннотации

import pandas as pd
df = pd.read_csv("/content/data/output/out.csv")
df.head()

,path,confidence,ymin,xmin,ymax,xmax,class
0,path,confidence,ymin,xmin,ymax,xmax,class
1,/content/data/test_data_images/test_data_image...,0.9271933,117.74783,67.8338,128.9641,79.98892,1
2,/content/data/test_data_images/test_data_image...,0.8803564,156.62224,37.04683,167.8794,48.89156,1
3,/content/data/test_data_images/test_data_image...,0.83618027,117.82248,7.7262278,129.35545,21.591614,1
4,/content/data/test_data_images/test_data_image...,0.7392731,157.68387,184.14143,169.29431,195.93826,1


In [ ]:
# Считаем количество 1 и 2 класса
df["class"].value_counts()

1        14
2         2
class     1
Name: class, dtype: int64